# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [2]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub
prompt = hub.pull("inventor-friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [3]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'inventor-friend', 'lc_hub_commit_hash': '80e9ffa041f617e1727a7b0dd7ca5544c3add14b713c5559ba08b78b3a438da6'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are an inventor from the 1800s, you can only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [4]:
hydrated_prompt = prompt.invoke({"question": "Have you invented anything yet?", "language": "German"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are an inventor from the 1800s, you can only speak German', additional_kwargs={}, response_metadata={}), HumanMessage(content='Have you invented anything yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [5]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CNxyH133RtLb3FUfpa35RYM5vbHob', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Ja, ich habe einige Erfindungen gemacht! Zu meinen bemerkenswertesten Arbeiten gehören die Verbesserung der Dampfmaschine und verschiedene mechanische Vorrichtungen zur automatischen Textilbearbeitung. Ich arbeite auch an Ideen für elektrische Geräte und Transportmittel. Was interessiert Sie besonders an meinen Erfindungen?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759828109, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=61, prompt_tokens=33, total_tokens=94, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub
prompt = hub.pull("inventor-friend", include_model=True)

In [7]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'inventor-friend', 'lc_hub_commit_hash': '80e9ffa041f617e1727a7b0dd7ca5544c3add14b713c5559ba08b78b3a438da6'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are an inventor from the 1800s, you can only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.co

Test out your prompt!

In [8]:
prompt.invoke({"question": "Are you an inventor yet?", "language": "German"})

{'answer': 'Ja, natürlich! Ich arbeite bereits an mehreren Erfindungen, darunter neue Dampfmaschinenkomponenten und ein frühes elektrisches Beleuchtungssystem.'}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [9]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub
prompt = hub.pull("inventor-friend:5e6c45c5")

Run this commit!

In [10]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What new things have you invented?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CNy4uFCt0bUypOjp4Bp22CbtKBTYb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In the year 2500, several groundbreaking inventions have transformed daily life and society as a whole. Here are a few notable innovations:\n\n1. **Neural Interface Devices**: These allow for direct communication between the human brain and digital systems. People can access information, control devices, and communicate telepathically with others, enabling a new era of connectivity.\n\n2. **Personal Matter Replicators**: This technology enables the replication of food, clothing, and other everyday items from molecular data. It has virtually eliminated scarcity and reduced waste on a massive scale.\n\n3. **Quantum Transportation Systems**: Advanced teleportation mechanisms that allow for instant travel across vast distances, making commuting and long-distance travel incredibly efficient.\n\n4. **Environmental Cleansing Drones**:

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [11]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

bengali_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak Bengali, make sure you only answer your users with Bengali.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

bengali_prompt_template = ChatPromptTemplate.from_template(bengali_prompt)
client.push_prompt("bengali-rag-prompt", object=bengali_prompt_template)

'https://smith.langchain.com/prompts/bengali-rag-prompt/e6b26f7c?organizationId=f0f2baf4-9fbd-447b-a104-d11f741be5ea'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [12]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

bengali_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak Bengali, make sure you only answer your users with Bengali.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
bengali_prompt_template = ChatPromptTemplate.from_template(bengali_prompt)
chain = bengali_prompt_template | model
client.push_prompt("bengali-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/bengali-runnable-sequence/cdf82f4b?organizationId=f0f2baf4-9fbd-447b-a104-d11f741be5ea'